# How to describe smoothnes of a slip vector along fault elements?

## Planar faults
The question is motivated by needing to think about implementing continuity and smoothness of slip for non-planar faults. For a scalar value of slip on planar faults, this is trivial since we only need to enforce the following equation 
$\begin{align}
s(\zeta_{n_3})\vert_{(k)} = s(\zeta_{n_1})\vert_{(k+1)} \\
\frac{\partial s}{\partial \zeta}(\zeta_{n_3})\vert_{(k)} = \frac{\partial s}{\partial \zeta}(\zeta_{n_1})\vert_{(k+1)}
\end{align}$ 
Here $\zeta$ is a variable describing distance along the fault element, $n_3\vert_{(k)},n_1\vert_{(k+1)}$ refers to the $3^{rd}$ node of the $k^{th}$ element and $1^{st}$ node of the $k+1^{th}$ element respectively. Even if slip is a vector, the above equations can be applied independently for each slip component.

## Slip on an arbitrary oriented fault element
Consider a fault element with two unit vectors $\hat{d} = \begin{bmatrix} d_x \\ d_z \end{bmatrix},\hat{n}= \begin{bmatrix} n_x \\ n_z \end{bmatrix}$ that describe the shear and normal directions respectively. The slip vector in terms of the global coordinates is
$\begin{equation}\underline{s} = \begin{bmatrix} s_x \\ s_z \end{bmatrix}\end{equation}$ 
In terms of slip quantities local to the fault element, $s_s,s_n$ are shear and tensile slip respectively. The slip vector is then,
$\begin{equation}\underline{s} = \begin{bmatrix} s_s d_x + s_n n_x \\ s_s d_z + s_n n_z \end{bmatrix}\end{equation}$
Maintaining continuity of the slip vector for arbitrary intersections of fault elements simply requires us to ensure the above equation (in a global coordinate system) is satisfied at an overlapping node.

### Slip gradient for a simple case
Slip gradients are a bit more complicated. For clarity, let us start with an elementary case where the fault element is oriented along $\hat{d}= \begin{bmatrix} 1 \\ 0 \end{bmatrix}$ and the $\hat{n}= \begin{bmatrix} 0 \\ -1 \end{bmatrix}$. In such a case, the slip vector $\underline{s} = \begin{bmatrix} s_x \\ s_z \end{bmatrix}$ where $s_x = s_s, s_z = s_n$ and $s_s,s_n$ are shear and tensile slip respectively. The gradient of this vector is,

$\begin{equation} \nabla\underline{s} = 
\begin{bmatrix} \frac{\partial s_x}{\partial x} & \cancel{\frac{\partial s_x}{\partial z}} \\ \frac{\partial s_z}{\partial x} & \cancel{\frac{\partial s_z}{\partial z}}\end{bmatrix} 
= \begin{bmatrix}\frac{\partial s_s}{\partial \zeta} & 0 \\ \frac{\partial s_n}{\partial \zeta} & 0\end{bmatrix} \end{equation}$

The above jacobian has only 2 non-zero entries, because for this geometry $\frac{\partial}{\partial z} = 0$. 

### Slip gradient for arbitrary geometry
We can now take this above calculated jacobian and rotate it by an appropriate angle to obtain the slip gradient for any arbitrary geometry. We first write the rotation matrix in terms of the two orthonormal unit vectors $\hat{d},\hat{n}$,
$\begin{equation} R = \begin{bmatrix} d_x & d_z \\ n_x & n_z \end{bmatrix}\end{equation}$

The rotated slip gradient is now,

$\begin{aligned} R^T (\nabla\underline{s}) R & = \begin{bmatrix} d_x & n_x \\ d_z & n_z \end{bmatrix}\begin{bmatrix} \frac{\partial s_s}{\partial \zeta} & 0 \\ \frac{\partial s_n}{\partial \zeta} & 0\end{bmatrix}\begin{bmatrix} d_x & d_z \\ n_x & n_z \end{bmatrix} \\
& = \begin{bmatrix} \frac{\partial s_s}{\partial\zeta}d_x^2 + \frac{\partial s_n}{\partial\zeta}d_x n_x & 
\frac{\partial s_s}{\partial\zeta}d_x d_z + \frac{\partial s_n}{\partial\zeta}n_x d_z \\
\frac{\partial s_s}{\partial\zeta}d_x d_z + \frac{\partial s_n}{\partial\zeta}n_z d_x & 
\frac{\partial s_s}{\partial\zeta}d_z^2 + \frac{\partial s_n}{\partial\zeta}n_z d_z
\end{bmatrix}\end{aligned}$

The quantity that we actually care about to enforce smoothness of slip or continuity of the gradient of slip is a directional derivative of the slip vector along the fault element i.e., 

$\begin{aligned} \nabla_{\zeta}\underline{s} &= \left[R^T (\nabla\underline{s}) R\right] . \begin{bmatrix} d_x \\ d_z \end{bmatrix} \\
&= \begin{bmatrix} \frac{\partial s_s}{\partial\zeta}d_x(d_x^2 + d_z^2) + \frac{\partial s_n}{\partial\zeta}n_x(d_x^2 + d_z^2) \\ \frac{\partial s_s}{\partial\zeta}d_z(d_x^2 + d_z^2) + \frac{\partial s_n}{\partial\zeta}n_z(d_x^2 + d_z^2) \end{bmatrix}
\end{aligned}$

Considering $\Vert\hat{d}\Vert = 1$, the directional derivative reduces to
$\begin{equation} \nabla_{\zeta}\underline{s} = \begin{bmatrix} \frac{\partial s_s}{\partial\zeta}d_x + \frac{\partial s_n}{\partial\zeta}n_x \\ \frac{\partial s_s}{\partial\zeta}d_z + \frac{\partial s_n}{\partial\zeta}n_z \end{bmatrix} \end{equation}$
Now we have a framework for enforcing smoothness of the slip vector for an arbitrary fault geometry, which is what is implemented in [the previous notebook on non-planar patches](Nonplanarpatches_rotation.ipynb). 

An alternative to the above methodology of enforcing slip smoothness would be to state that the traction vector from slip of the $k^{th}$ element evaluated at its $3^{rd}$ node is equal to the traction vector from slip of the $k+1^{th}$ element evaluated at its $1^{st}$ node. This is possibly equivalent to the analytical derivation because the traction vector $\underline{t} = \begin{bmatrix} \sigma_{xx}n_x + \sigma_{xz}n_z \\ \sigma_{xz}n_x + \sigma_{zz}n_z\end{bmatrix}$, and we know that components of the Cauchy stress tensor evaluated exactly on the fault are functions of the slip vector and the related gradients. Enforcing continuity of tractions might just be the same as enforcing continuity of slip gradients at any node? However, implementing this might not be possible because computing the Greens functions for tractions at an end node requires us to evaluate a singular kernel and so the analytical formulation is likely to be a more stable form. 

Additionally, the traction representation illuminates why we cannot expect stresses to be continuous in space for non-planar faults. This is because we can only enforce the traction vector, which is a projection of the full stress tensor onto the orientation of the fault element and thus a lower dimensional object, to be continuous. For the case of planar faults, continuity of tractions at a given node guaranteed continuity of the stress tensor because the projection vector (unit normal) was identical on either side of that node.
